In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(1)

In [2]:
cifar100_args = dotdict()

cifar100_args.name = 'cifar100'
cifar100_args.train_batch_size = 128
cifar100_args.predict_batch_size = 128
cifar100_args.device = ['cuda:1', 'cuda:6']
cifar100_args.train_epochs = 250
cifar100_args.lr = 0.01
cifar100_args.criterion = 'CE'
cifar100_args.optimizer = 'AdamW'
cifar100_args.scheduler = 'multistep'
cifar100_args.attack = {'fgsm':(0.005,), 'pgd':(0.005,0.1,20)}

activation = 'relu'
input_channel = 3
knpp = [48,96,144,240,336,432,528]

cifar100_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atrc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('atrc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atrc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('atrc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atrc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (2,2), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
    ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 100, (1,2,3)))
]

In [3]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train_batch_size = 128
cifar10_args.predict_batch_size = 128
cifar10_args.device = ['cuda:1', 'cuda:6']

cifar10_args.train_epochs = 250
cifar10_args.lr = 0.01
cifar10_args.criterion = 'CE'
cifar10_args.optimizer = 'AdamW'
cifar10_args.scheduler = 'OneCycle'
cifar10_args.attack = {'fgsm':(0.005,), 'pgd':(0.005,0.1,20)}

activation = 'relu'
input_channel = 3
# knpp = [32,64,96,128,160,192,224]

# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
#     ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (2,2), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
# #     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3)))\
# ]


# knpp = [64,96,128,192,256,384,512]
# groups=32
knpp = [48,96,144,240,336,432,528]
groups=48

cifar10_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, groups), 1, 'first', (2,2), activation, False)),
    ('atrc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, groups), 1, 'first', (2,2), activation, False)),
    ('atrc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, groups), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, groups), 1, 'first', (2,2), activation, False)),
    ('linear', (knpp[6], knpp[6], 1, 1)),
#     ('atrc2d', ((knpp[6], knpp[7], (3,3), 1, 1, 1, groups), 1, None, None, activation, True)),
    ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 10, (1,2,3)))
]

In [ ]:
driver = ABC_Driver(cifar10_args, None, record_path=None, if_hash=False)
driver.train()

Use: ['cuda:1', 'cuda:6']
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
add record: 03/26/2023 00:52
epoch: 0, train_loss: 1.8333, test_metric: 0.492, time: 428.7156186103821
epoch: 1, train_loss: 1.5228, test_metric: 0.5935, time: 425.42906880378723
epoch: 2, train_loss: 1.3636, test_metric: 0.6443, time: 425.3009145259857
epoch: 3, train_loss: 1.2362, test_metric: 0.6824, time: 425.7143015861511


In [4]:
# torch.save(driver.model.state_dict(), "save/CIFAR100_ABCCNN_2023_03_13.pt")